# LLMOPS
1. Load Data / Processing / Embeddings
2. Model Selection / Load LLM / Fine-Tunning / Prompt Engineering
3. LLM Chain and Agents
4. Evaluate / Metrics
5. Deployment / API Gateaway
6. Monitoring

## Load Data / Pre-Processing / Embeddings

### Load Data

In [1]:
import os
from langchain.document_loaders import PDFPlumberLoader

#File path with the PDF that I will use for testing
file_path = r'documents\SIGNED CompanyB_SOW_Tableau Resouces IT Q2-Q4 2023_1 April 2024 1.pdf'
file_name = os.path.splitext(file_path)[0].split('/')[-1]

loader = PDFPlumberLoader(file_path)
documents = loader.load()

### Pre Processing

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
    )

chunks = text_splitter.split_documents(documents)

### Embeddings

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

model_name = 'BAAI/bge-large-en-v1.5'
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(documents = chunks, embedding = embedding_model)
vectordb.save_local(f'embeddings'/{file_name}')

## Load LLM / Prompt Engineering

### Model Selection and Load 

In [9]:
from langchain_community.document_loaders.base import BaseLoader
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.documents import Document
from langchain_core.language_models.llms import BaseLLM
from langchain_core.outputs import Generation, LLMResult
from langchain_core.pydantic_v1 import Field, root_validator
from typing import Any, Dict, Iterator, List, Optional
import requests

class CustomLLM(BaseLLM):
    def __init__(self):
        super(CustomLLM, self).__init__()
        self.callbacks=None
        self.verbose=False
        self.tags=None
        self.metadata = None
        self.cache= None

    @property
    def _default_params(self) -> Dict[str, Any]:
        """Get the default parameters for calling vllm."""
        return {}

    @root_validator(allow_reuse=True)
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that python package exists in environment."""

        return {}
    
    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "vllm"

    def _generate(
        self,
        prompts: List[str],
        params: dict ={'temperature':0.7, 'top_p':1.0, 'max_tokens':200, 'skip_special_tokens':True,'stop':['Note','Please'],'frequency_penalty':1.2},        
        **kwargs: Any,
    ) -> LLMResult:
        """Run the LLM on the given prompt and input."""
        
        # call the model
        response = requests.post(
            "http://3.108.133.55:8502/llama_generate",
            json={
                "prompt": prompts,
                "kwargs":params
            },
        )
        outputs = response.json()
  
        generations = []
        for output in outputs:
            text = output['outputs'][0]['text']
            generations.append([Generation(text=text)])

        return LLMResult(generations=generations)

In [23]:
llm = CustomLLM()

### Prompt Engineering

In [24]:
from langchain.prompts import PromptTemplate

template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:"""

prompt = PromptTemplate(
    input_variables=["question", "context"], template=template
)

## LLM Chain

In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import LLMChain, RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = CustomLLM()
#This was the query used to extract the context needed

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectordb.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Deployment

In [ ]:
def chatbot(question:str)->str:
    answer=rag_chain.invoke(question)
    print(answer)
    return answer

In [34]:
response=chatbot("Which are the companies between this contract?")

 Based on the provided context, the companies involved in this contract are:

1. Company C LLP
2. Company B Limited
